<a href="https://colab.research.google.com/github/wlgns410/pandas/blob/main/3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
cd drive/MyDrive/drive/MyDrive/RecoSys/Data

[Errno 2] No such file or directory: 'drive/MyDrive/drive/MyDrive/RecoSys/Data'
/content/drive/MyDrive/drive/MyDrive/RecoSys/Data


In [8]:
base_src = '/content/drive/MyDrive/drive/MyDrive/RecoSys/Data'

u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src, sep='|', names=u_cols, encoding='latin-1')

u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release date', 'video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama',
          'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
          'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_src, sep='|', names=i_cols, encoding='latin-1')

u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t', names=r_cols, encoding='latin-1')

# ratings DataFrame에서 timestamp 제거
ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

x = ratings.copy()
y = ratings['user_id']

### test_size: 25%, train:75%로 구성하겠다는 뜻  stratify는 데이터 뭉침을 방지 위함
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, stratify=y)

# 정확도(RMSE) 계산 함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) **2))

# 모델별 RMSE 계산하는 함수
def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

# best_Seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(['movie_id'])['rating'].mean()

def best_seller(user_id, movie_id):
  try:
    rating = train_mean[movie_id]

  except:
    rating = 3.0
  return rating

score(best_seller)

merged_ratings = pd.merge(x_train,users)

users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

### 코사인 유사도 계산###
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)


In [9]:
## 주어진 영화의 가중평균을 계산하는 함수
def CF_simple(user_id, movie_id):
  if movie_id in rating_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores,movie_ratings) / sim_scores.sum()

  else:
    mean_rating = 3.0
  return mean_rating




In [22]:
user_similarity

array([[1.        , 0.0947917 , 0.03162495, ..., 0.06377042, 0.12666333,
        0.32161978],
       [0.0947917 , 1.        , 0.08770493, ..., 0.07827937, 0.11258756,
        0.08543742],
       [0.03162495, 0.08770493, 1.        , ..., 0.02444529, 0.10442267,
        0.        ],
       ...,
       [0.06377042, 0.07827937, 0.02444529, ..., 1.        , 0.06903738,
        0.04876658],
       [0.12666333, 0.11258756, 0.10442267, ..., 0.06903738, 1.        ,
        0.17530975],
       [0.32161978, 0.08543742, 0.        , ..., 0.04876658, 0.17530975,
        1.        ]])